In [43]:
import pandas as pd
from datetime import timedelta, date
from get_data import daterange, get_data, parse_info

In [40]:
cities = pd.read_csv('data/ongoing_cities.csv', header = False)

In [41]:
cities

,state_code,city_code,city
0,USA-MA,USA-BOS-BOT,Boston
1,USA-IL,USA-CHI-CHH,Chicago
2,USX-TX,USA-DAL-DLL,Dallas
3,USA-MI,USA-DET-DET,Detroit
4,USA-NV,USA-LAS-LAS,Las Vegas
5,USA-CA,USA-LAX-LSN,Los Angeles
6,USA-FL,USA-MIA-MIF,Miami
7,USA-MN,USA-MIN-MIA,Minneapolis
8,USA-NY,USA-NYM-NEY,New York City-Manhattan
9,USA-FL,USA-ORL-ORL,Orlando


In [3]:
url = 'https://s3-us-west-2.amazonaws.com/hoodsjson/USA-CA/USA-SFO-SNF/2014-09-30/0.html'

In [4]:
json_df = pd.read_json(url)    

In [5]:
condition = json_df['source'] == 'CRAIG'
craig_df = json_df[condition]
craig_df = craig_df.reset_index()
del craig_df['index']

In [6]:
cols = ['heading', 'body', 'price', 'lat', 'long', 'region', 'neighborhood', 
        'beds', 'baths', 'parking', 'washer_dryer']

In [7]:
results_df = parse_info(craig_df, headers = cols)

In [9]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498 entries, 0 to 497
Data columns (total 12 columns):
heading         498 non-null object
body            498 non-null object
price           495 non-null float64
lat             498 non-null object
long            498 non-null object
region          497 non-null object
neighborhood    490 non-null object
beds            497 non-null object
baths           465 non-null object
parking         498 non-null object
washer_dryer    498 non-null object
date            498 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(10)
memory usage: 50.6+ KB


In [11]:
start = date(2014, 9, 28)
end = date(2014, 9, 30)

In [44]:
get_data(start, end)

TypeError: parser_f() got an unexpected keyword argument 'headers'